# Project B


This notebook will cover only cover the data prep and merge of the two larger json files.

## Packages Needed

In [1]:
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
import os

In [2]:
path = 'C:\\Users\\Miche\\OneDrive - Danmarks Tekniske Universitet\\MMC\\2. Semester\\Social Data\\websites\\data'
os.chdir(path)
os.getcwd()

'C:\\Users\\Miche\\OneDrive - Danmarks Tekniske Universitet\\MMC\\2. Semester\\Social Data\\websites\\data'

## Load Data Business Dataset + Prep

In [3]:
df = pd.read_json('yelp_academic_dataset_business.json', lines=True)
print(df.shape)

(160585, 14)


In [4]:
df.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

We only want to look at the restaurants that are open and not closed.

In [5]:
df = df[df['is_open']==1]

In [6]:
df = df.drop(['is_open'], axis=1)

Start by select all the rows that mention restaurants.

In [7]:
df = df[df['categories'].str.contains('Restaurants',
              case=False, na=False)]

In [8]:
df_explode = df.assign(categories = df.categories
                         .str.split(', ')).explode('categories')

In [9]:
df_explode.categories.value_counts().head(20)

Restaurants                  32022
Food                         10732
Nightlife                     5550
Bars                          5345
Sandwiches                    4795
American (Traditional)        4379
Fast Food                     4326
Pizza                         3890
Breakfast & Brunch            3872
American (New)                3254
Burgers                       3246
Coffee & Tea                  3010
Mexican                       2815
Chinese                       2329
Italian                       2130
Seafood                       1998
Salad                         1975
Event Planning & Services     1916
Cafes                         1871
Japanese                      1773
Name: categories, dtype: int64

In [10]:
remove = ['Food Delivery Services','Food Safety Training', 'Food Tours', 
          'Food Banks','Chinese Martial Arts','Traditional Chinese Medicine',
         'Coffee & Tea Supplies','Japanese Curry', 'Food Court',
         'Food Trucks','Food Stands']

df = df[~df['categories'].str.contains('|'.join(remove),
              case=False, na=False)]

Try to split the category column into different subgroups, so we can do some nice plotting.

In [11]:
cat_kitchens = ['thai', 'chinese','japanese','korean','indian','american',
                 'caribbean','italian','mediterranean','mexican', 'cajun',
                'vietnamese','greek']

cat_type = ['Food','Nightlife','Bars','Sandwiches','Pizza','Breakfast & Brunch', 'Fast Food',
            'Burgers','Salad', 'Buffet', 'Cafes','Coffee & Tea', 'Vegetarian', 'Steakhouse', 'Sushi Bars',
            'Diners','Wine Bars']

In [12]:
df_explode = df.assign(categories = df.categories
                         .str.split(', ')).explode('categories')

In [13]:
df_cat_kitchen = df_explode[df_explode['categories'].str.contains(
              '|'.join(cat_kitchens),
              case=False, na=False)]

df_cat_kitchen = df_cat_kitchen.rename(columns={'categories':'cat_kitchen'})
df_cat_kitchen = df_cat_kitchen['cat_kitchen']

In [14]:
df_cat_type = df_explode[df_explode['categories'].str.match(
              '|'.join(cat_type),
              case=True, na=False)]

df_cat_type = df_cat_type.rename(columns={'categories':'cat_type'})
df_cat_type = df_cat_type['cat_type']

In [15]:
df_cat_type.unique()

array(['Food', 'Bars', 'Nightlife', 'Salad', 'Sandwiches', 'Cafes',
       'Vegetarian', 'Breakfast & Brunch', 'Pizza', 'Sushi Bars',
       'Steakhouses', 'Fast Food', 'Burgers', 'Coffee & Tea', 'Diners',
       'Buffets', 'Wine Bars'], dtype=object)

In [16]:
df_merge_list = [df, df_cat_kitchen, df_cat_type]

In [17]:
df_merged = reduce(lambda  left, right: pd.merge(left,right,left_index=True,
                                            right_index=True), df_merge_list)

In [18]:
df_merged.shape

(29836, 15)

In [19]:
df_merged.state.unique()

array(['CO', 'FL', 'MA', 'OR', 'OH', 'GA', 'TX', 'BC', 'WA', 'KS', 'KY'],
      dtype=object)

Below is a for-loop and if-statement, this is used to insert the full name of the state. It is assumed that not all people know what state MA is. Furthermore, the latitude and longitude for each state is also inserted. These are used later to give us an overview over where we have data from.

In [20]:
name = []
long = []
lat = []
for state in df_merged['state']:
    if state =='MA':
        name.append('Massachusetts')
        lat.append(42.407211)
        long.append(- 71.382439)
    elif state =='FL':
        name.append('Florida')
        lat.append(27.994402)
        long.append(-81.760254)
    elif state =='OR':
        name.append('Oregon')
        lat.append(44.000000)
        long.append(-120.500000)
    elif state =='BC':
        name.append('British Columbia')
        lat.append(53.726669)
        long.append(-127.647621)
    elif state =='GA':
        name.append('Georgia')
        lat.append(33.247875)
        long.append(-83.441162)
    elif state =='TX':
        name.append('Texas')
        lat.append(31.000000)
        long.append(-100.000000)
    elif state =='OH':
        name.append('Ohio')
        lat.append(40.367474)
        long.append(-82.996216)
    elif state =='CO':
        name.append('Colorado')
        lat.append(39.113014)
        long.append(-105.358887)
    elif state =='WA':
        name.append('Washington')
        lat.append(47.751076)
        long.append(-120.740135)
    elif state =='KS':
        name.append('Kansas')
        lat.append(38.500000)
        long.append(-98.000000)
    elif state == 'KY':
        name.append('Kentucky')
        lat.append(37.839333)
        long.append(-84.270020)
        
df_merged['state_name'], df_merged['latitude_state'], df_merged['longitude_state']  = name, lat, long

Extracting some more features, e.g. pricerange on the restaurants.

In [21]:
df_merged['attributes'] = df_merged['attributes'].apply(lambda x: {} if x is None else x)
df_att = pd.json_normalize(df_merged.attributes)
df_att = df_att['RestaurantsPriceRange2'].to_frame()

In [22]:
PriceRange = []
AvgPrice = []
for row in df_att['RestaurantsPriceRange2']:
    if row == '1' :    
        PriceRange.append('$')
        AvgPrice.append(1)
    elif row == '2':   
        PriceRange.append('$$')
        AvgPrice.append(20)
    elif row == '3':  
        PriceRange.append('$$$')
        AvgPrice.append(45)
    elif row == '4':  
        PriceRange.append('$$$$')
        AvgPrice.append(75)
    else:           
        PriceRange.append('Unknown')A
        AvgPrice.append(0)
        
df_merged['PriceRange'], df_merged['AvgPrice'] = PriceRange, AvgPrice
df_merged.shape

(29836, 20)

In [23]:
df_merged['PriceRange'].value_counts()

$$         17575
$           8268
Unknown     2625
$$$         1223
$$$$         145
Name: PriceRange, dtype: int64

## Load Review Data (Massive dataset)

In [24]:
size = 500000
review = pd.read_json('yelp_academic_dataset_review.json', lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [25]:
# There are multiple chunks to be read
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(df_merged, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

416263 out of 500,000 related reviews
384085 out of 500,000 related reviews
371677 out of 500,000 related reviews
358510 out of 500,000 related reviews
349565 out of 500,000 related reviews
356785 out of 500,000 related reviews
353610 out of 500,000 related reviews
407361 out of 500,000 related reviews
371185 out of 500,000 related reviews
362734 out of 500,000 related reviews
361974 out of 500,000 related reviews
362709 out of 500,000 related reviews
388455 out of 500,000 related reviews
373812 out of 500,000 related reviews
414622 out of 500,000 related reviews
414618 out of 500,000 related reviews
433111 out of 500,000 related reviews
107384 out of 500,000 related reviews


In [26]:
df.shape

(6588460, 24)

## Load UserId Data (Massive dataset)

In [27]:
size = 500000
review = pd.read_json('yelp_academic_dataset_user.json', lines=True,
                      dtype={'user_id':str,'name':str,
                             'review_count':int},
                      chunksize=size)

In [28]:
# There are multiple chunks to be read
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['useful','elite','funny','cool','friends','yelping_since',
                                      'fans','compliment_more','compliment_profile','compliment_hot',
                                      'compliment_more', 'compliment_profile', 'compliment_cute',
                                      'compliment_list', 'compliment_note', 'compliment_plain',
                                      'compliment_cool', 'compliment_funny', 'compliment_writer',
                                      'compliment_photos'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'name': 'username','review_count':'user_count'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(df, chunk_review, on='user_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

4073724 out of 500,000 related reviews
1175245 out of 500,000 related reviews
689031 out of 500,000 related reviews
486388 out of 500,000 related reviews
164072 out of 500,000 related reviews


In [29]:
df.shape

(6588460, 27)

In [30]:
df.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'attributes',
       'categories', 'hours', 'cat_kitchen', 'cat_type', 'state_name',
       'latitude_state', 'longitude_state', 'PriceRange', 'AvgPrice',
       'user_id', 'review_stars', 'text', 'date', 'username', 'user_count',
       'average_stars'],
      dtype='object')

## More data prep

Select 2014-2021, so we have full years

In [31]:
print (df.date.min())
print (df.date.max())

2004-10-14 02:57:52
2021-01-28 15:23:52


In [32]:
df.date.dtypes

dtype('O')

In [33]:
df['date'] = pd.to_datetime(df['date'])

In [34]:
df.date.dtypes

dtype('<M8[ns]')

In [35]:
time_start = pd.Timestamp(2014,1,1)

In [36]:
#Filter the data
df = df[(df['date']>= time_start)]

In [37]:
print (df.date.min())
print (df.date.max())

2014-01-01 00:01:51
2021-01-28 15:23:52


In [38]:
df.shape

(5177322, 27)

Save to csv file

In [39]:
path = 'C:\\Users\\Miche\\OneDrive - Danmarks Tekniske Universitet\\MMC\\2. Semester\\Social Data\\websites\\data'
os.chdir(path)
os.getcwd()

'C:\\Users\\Miche\\OneDrive - Danmarks Tekniske Universitet\\MMC\\2. Semester\\Social Data\\websites\\data'

Split the dataframe into equal big parts!

In [40]:
df_2014 = df[(df['date'].dt.year == 2014)]
df_2015 = df[(df['date'].dt.year == 2015)]
df_2016 = df[(df['date'].dt.year == 2016)]
df_2017 = df[(df['date'].dt.year == 2017)]
df_2018 = df[(df['date'].dt.year == 2018)]
df_2019 = df[(df['date'].dt.year == 2019)]
df_2020 = df[(df['date'].dt.year >= 2020)]

In [41]:
df_2014.to_csv("yelp_reviews_RV_categories_2014.csv", index=False)
df_2015.to_csv("yelp_reviews_RV_categories_2015.csv", index=False)
df_2016.to_csv("yelp_reviews_RV_categories_2016.csv", index=False)
df_2017.to_csv("yelp_reviews_RV_categories_2017.csv", index=False)
df_2018.to_csv("yelp_reviews_RV_categories_2018.csv", index=False)
df_2019.to_csv("yelp_reviews_RV_categories_2019.csv", index=False)
df_2020.to_csv("yelp_reviews_RV_categories_2020.csv", index=False)